In [1]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '2.1.1'


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
        H = fromCopyString(variant[3])
    return A, B, L, H

In [9]:
A, B, L, H = получить_матрицы_варианта(8)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
H = sympy.Matrix(H)

In [10]:
def матрица_управляемости(A, B, L):
    матрица = sympy.eye(3)
    матрица[:, 0] = B 
    матрица[:, 1] = A*B
    матрица[:, 2] = A**2*B
    return матрица

In [11]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[ 5, -32,  284],
[-5,  -4,  184],
[ 4,  -4, -320]])

In [12]:
def определитель_матрицы_управляемости(S):
    определитель = S.det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [13]:
print(определитель_матрицы_управляемости(S))

47952


In [14]:
def коэффициенты_характеристического_полином_разомкнутой_системы(A):
    l = sympy.symbols('lambda')
    I = sympy.eye(3)
    detChPolyOpen = (l*I - A).det()
    c0 = detChPolyOpen.coeff(l, 0)
    c1 = detChPolyOpen.coeff(l, 1)
    c2 = detChPolyOpen.coeff(l, 2)
    ChPolyOpen = sympy.Matrix([
        c0,
        c1,
        c2
        ])
    return ChPolyOpen

In [15]:
коэффициенты_характеристического_полином_разомкнутой_системы(A)

Matrix([
[372],
[145],
[ 17]])

In [16]:
def коэффициенты_характеристического_полином_замкнутой_системы(L):
    c0 = L[0]*L[1]*L[2]
    c1 = L[0]*L[1] + L[1]*L[2] + L[0]*L[2]
    c2 = L[0]+L[1]+L[2]

    ChPolyOpen = sympy.Matrix([
        -c0,
        c1,
        -c2
        ])
    return ChPolyOpen

In [17]:
коэффициенты_характеристического_полином_замкнутой_системы(L)

Matrix([
[160],
[ 92],
[ 17]])

In [23]:
def матрица_D_H(A, B, L, H):
    c0, c1, c2 = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    k0, k1, k2 = коэффициенты_характеристического_полином_замкнутой_системы(L)
    d3 = B
    d2 = A*d3 + c2*B
    d1 = A*d2 + c1*B
    res = sympy.eye(3)
    res[:, 0] = d1
    res[:, 1] = d2
    res[:, 2] = d3
    return (res.T*H.T)

In [30]:
rs = матрица_D_H(A, B, L, H)
rs

Matrix([
[465, -609],
[ 53,  -89]])

In [25]:
def вектор_из_коэффициентов(A, B, L, H):
    return коэффициенты_характеристического_полином_разомкнутой_системы(A) - коэффициенты_характеристического_полином_замкнутой_системы(L)

In [47]:
вектор_из_коэффициентов(A, B, L, H)[:2, 0]

Matrix([
[212],
[ 53]])

In [60]:
def матрица_G(A, B, L, H):

    res = (матрица_D_H(A, B, L, H)[:2, :].inv())*(вектор_из_коэффициентов(A, B, L, H))[:2, 0]

    return res.n().T

In [53]:
res = матрица_G(A, B, L, H)
res

Matrix([[-1.47222222222222, -1.47222222222222]])

In [55]:
toCopyString(res)

'matrix([-1.47222222222222,-1.47222222222222])'

In [59]:
for var_number in range(1, 31):
    A, B, L, H = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    H = sympy.Matrix(H)
    
    S                        = матрица_управляемости(A, B, L)
    определитель             = определитель_матрицы_управляемости(S)
    коэффициенты_разомкнутой = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    коэффициенты_замкнутой   = коэффициенты_характеристического_полином_замкнутой_системы(L)
    DH_матрица               = матрица_D_H(A, B, L, H)
    G                        = матрица_G(A, B, L, H)
    
    with open(f'./results/{var_number}.txt', 'w') as file:
        file.write(toCopyString(S) + '\n')
        file.write(str(определитель) + '\n')
        file.write(toCopyString(коэффициенты_разомкнутой) + '\n')
        file.write(toCopyString(коэффициенты_замкнутой) + '\n')
        file.write(toCopyString(DH_матрица) + '\n')
        file.write(toCopyString(G) + '\n')
        